<a href="https://colab.research.google.com/github/VladShajdulin/OTUS_project/blob/main/OTUS_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import json
import regex
import torch
from torch import Tensor
from sentence_transformers import CrossEncoder
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import mine_hard_negatives
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.cross_encoder.losses import BinaryCrossEntropyLoss
from sentence_transformers.cross_encoder import CrossEncoderTrainingArguments, CrossEncoderTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
from datasets import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
torch.manual_seed(142)
path = '/content/drive/MyDrive/Colab Notebooks/OTUS_project/articles_links.json'
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else 'cpu'
print(device)

cuda


In [ ]:
all_data = pd.read_json(path)
df = all_data[['title', 'link', 'annot', 'tags']]
df

,title,link,annot,tags
0,"Ученые обнаружили еще одну причину, по которой...",https://naked-science.ru/article/biology/uchen...,Голые землекопы живут почти 40 лет — невероятн...,"[Биология, голые землекопы, ДНК, долголетие]"
1,"Бактерии создали из себя батарейку, чтобы разр...",https://naked-science.ru/article/biology/bacte...,Они использовали содержащееся в почве железо. ...,"[Биология, аккумулятор, бактерии, биопленки]"
2,Бактерии и добавки пероксида кальция сделают д...,https://naked-science.ru/article/column/bakter...,Ученые РГУ нефти и газа (НИУ) имени И.М. Губки...,"[Губкинский университет, арктика, бактерии, ка..."
3,Российские физики предложили искать темную мат...,https://naked-science.ru/article/column/temnuy...,Большой коллектив российских ученых из ведущих...,"[ФизТех, вселенная, квантование гравитации, те..."
4,Белорусские ученые вырастили кожу из грибов и ...,https://naked-science.ru/article/column/beloru...,Специалисты UST Inc. разработали технологию по...,"[Unitsky String Technologies Inc., биоматериал..."
...,...,...,...,...
8267,В Испании нашли свой Стоунхендж,https://naked-science.ru/article/anthropology/...,"На поле, предназначенном для выращивания авока...","[Антропология, археология, древние люди, Испания]"
8268,Астрономы оценили максимально возможное число ...,https://naked-science.ru/article/astronomy/mak...,"Ученые подсчитали, что без больших изменений о...","[Астрономия, естественный спутник, Луна, спутн..."
8269,Рецепт сознания: могут ли люди и роботы мечтат...,https://naked-science.ru/article/hi-tech/retse...,В XXI веке вопрос о природе сознания станет од...,"[Технологии, искусственный интеллект, машинное..."
8270,У берегов Западной Африки нашли древний ударны...,https://naked-science.ru/article/geology/udarn...,"Ученые допускают, что 66 миллионов лет назад с...","[Геология, астероид, гибель динозавров, диноза..."


In [ ]:
def clean_text(text, pattern=r'[^\w\p{P}]+'):
  pattern = regex.compile(pattern)
  text = pattern.sub(' ', text)

  return regex.sub(r'\s+', ' ', text).strip()

In [ ]:
texts = [clean_text(t) for t in df['annot']]
passages = [f'passage: {t}' for t in texts]
texts[:5]

['Голые землекопы живут почти 40 лет — невероятный срок для грызуна размером с мышь. Недавнее открытие китайских ученых показало, что долголетие этим животным обеспечивает необычная версия одного белка, который не просто защищает организм, а буквально чинит его изнутри — ускоряет восстановление ДНК. Новые данные могут помочь продлить и человеческую жизнь.',
 'Они использовали содержащееся в почве железо. Днем бактерии захватывали фотоны, ночью — боролись с мешающими им жить соединениями.',
 'Ученые РГУ нефти и газа (НИУ) имени И.М. Губкина разработали новый метод обезвреживания нефтесодержащих отходов в Арктической зоне России. Он основан на комбинированном использовании бактериальных препаратов с добавками пероксида кальция, источника кислорода при обезвреживании отходов с использованием геотубных технологий. Технология успешно прошла полевые испытания.',
 'Большой коллектив российских ученых из ведущих физических институтов Москвы, Нижнего Новгорода, Сарова и Санкт-Петербурга предста

# 1. Семантический поиск
Для начала сделаем обычный семантический поиск на основе концепции "Retrieve & Re-rank". Посчитаем эмбеддинги для всех аннотаций статей, отберем из них топ 100, и затем посчитаем новые ранги для этого топа, в качестве реузьтата будет 10 наиболее релевантных статей.

In [ ]:
model = SentenceTransformer('d0rj/e5-base-en-ru', device=device)
embeddings = model.encode(
    passages,
    convert_to_tensor=True,
    show_progress_bar=True,
    device=device,
    normalize_embeddings=True
)
torch.save(embeddings, '/content/drive/MyDrive/Colab Notebooks/OTUS_project/embeddings.pt')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/529M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
embeddings = torch.load('/content/drive/MyDrive/Colab Notebooks/OTUS_project/embeddings.pt', map_location=device)
embeddings.shape

torch.Size([8272, 768])

In [ ]:
model = SentenceTransformer('d0rj/e5-base-en-ru', device=device)
reranker = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1', device=device)

def semantic_search(query, retriever, reranker, embeddings=embeddings, texts=texts, top_n=100, top_k=10, device=device):
  if isinstance(query, str):
    query = [query]

  n_q = len(query)
  pref_q = [f'query: {q}' for q in query]
  request = retriever.encode(
      pref_q,
      convert_to_tensor=True,
      device=device,
      normalize_embeddings=True
  )

  scores = request @ embeddings.T
  raiting = torch.argsort(scores, descending=True, dim=-1)

  if not reranker:
    raiting = raiting[:, :top_k].tolist()
    return raiting
  else:
    raiting = raiting[:, :top_n].tolist()
    documents = [[texts[i] for i in row] for row in raiting]
    pairs = [[(q, d) for d in row] for q, row in zip(query, documents)]

    a = reranker.predict(pairs[0], convert_to_tensor=True, batch_size=64)
    new_scores = torch.empty((n_q, top_n), dtype=a.dtype, device=device)
    new_scores[0, :] = a[:]

    for i in tqdm(range(1, len(pairs))):
      new_scores[i, :] = reranker.predict(pairs[i], convert_to_tensor=True, batch_size=64)[:]
    ranks = torch.argsort(new_scores, descending=True, dim=-1).tolist()
    new_rating = [[rai[i] for i in ran] for rai, ran in zip(raiting, ranks)]

    return [top[:top_k] for top in new_rating]

In [ ]:
query = ['стандартная модель', 'черные дыры', 'гравитационные волны']
search_result = semantic_search(query, retriever=model, reranker=reranker, top_k=10)[0]
df.iloc[search_result, :2]

  0%|          | 0/2 [00:00<?, ?it/s]

,title,link
6956,Ученые создали подробнейшую карту распределени...,https://naked-science.ru/article/astronomy/mat...
547,Масса нейтрино оказалась не связана с темной м...,https://naked-science.ru/article/physics/neutr...
5456,В МФТИ создали «Шамбалу» для данных о работе г...,https://naked-science.ru/article/column/v-mfti...
2909,Точная масса W-бозона согласуется со Стандартн...,https://naked-science.ru/article/physics/w-bos...
2845,Метод пермских ученых предсказал поведение сте...,https://naked-science.ru/article/column/metod-...
1137,Российские ученые научились справедливо делить...,https://naked-science.ru/article/column/sprave...
8002,Ученые разработали нейрокомпьютерную модель дл...,https://naked-science.ru/article/biology/uchen...
6221,Редкий распад бозона Хиггса может закрыть поис...,https://naked-science.ru/article/physics/raspa...
7793,В ЛЭТИ разработали модель для изучения магмы п...,https://naked-science.ru/article/column/v-leti...
5798,GPT-3 выполнил тест на мышление аналогиями луч...,https://naked-science.ru/article/hi-tech/gpt3-...


# 2. Оценка качества

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/OTUS_project/sinthetic_reqs.csv'
requests_df = pd.read_csv(path, index_col=0).dropna()
requests_df['id'] = requests_df['id'].map(json.loads)

train_req, test_req = train_test_split(
    requests_df,
    test_size=0.2,
    random_state=142,
    shuffle=True,
    stratify=requests_df['n']
)

In [ ]:
query = list(test_req['request'])
other_rr = CrossEncoder('DiTy/cross-encoder-russian-msmarco', device=device)

test_req['search_20_none'] = semantic_search(query, retriever=model, reranker=None, top_k=20)
test_req['search_10_none'] = test_req['search_20_none'].map(lambda x: x[:10])

test_req['search_20_rerank'] = semantic_search(query, retriever=model, reranker=reranker, top_k=20)
test_req['search_10_rerank'] = test_req['search_20_rerank'].map(lambda x: x[:10])

test_req['search_20_other'] = semantic_search(query, retriever=model, reranker=other_rr, top_k=20)
test_req['search_10_other'] = test_req['search_20_other'].map(lambda x: x[:10])

path = '/content/drive/MyDrive/Colab Notebooks/OTUS_project/req_with_search.csv'
test_req.to_csv(path, index=False)

  0%|          | 0/3598 [00:00<?, ?it/s]

  0%|          | 0/3598 [00:00<?, ?it/s]

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/OTUS_project/req_with_search.csv'
test_req = pd.read_csv(path)
test_req = test_req[~ test_req[['request', 'id']].duplicated()]
for col in ['id', 'search_10_none', 'search_20_none', 'search_10_rerank', 'search_20_rerank', 'search_10_other', 'search_20_other']:
  test_req[col] = test_req[col].map(json.loads)
test_req

,request,id,n,search_20_none,search_10_none,search_20_rerank,search_10_rerank,search_20_other,search_10_other
0,божьи черновики в геологии,[2536],1,"[328, 1785, 2768, 6511, 2989, 5484, 3785, 8006...","[328, 1785, 2768, 6511, 2989, 5484, 3785, 8006...","[1544, 8095, 2896, 6509, 5484, 60, 8072, 2683,...","[1544, 8095, 2896, 6509, 5484, 60, 8072, 2683,...","[2683, 1544, 5484, 8072, 7044, 5726, 517, 2238...","[2683, 1544, 5484, 8072, 7044, 5726, 517, 2238..."
1,связь между питанием и болезнью Альцгеймера,[687],1,"[4214, 687, 4566, 592, 6886, 6237, 1492, 2213,...","[4214, 687, 4566, 592, 6886, 6237, 1492, 2213,...","[687, 4739, 5007, 5161, 3822, 7225, 7904, 3877...","[687, 4739, 5007, 5161, 3822, 7225, 7904, 3877...","[6886, 7299, 687, 2094, 964, 4214, 279, 791, 6...","[6886, 7299, 687, 2094, 964, 4214, 279, 791, 6..."
2,особенности спаривания альпак,[3895],1,"[3895, 3400, 7863, 17, 5580, 3501, 6518, 4182,...","[3895, 3400, 7863, 17, 5580, 3501, 6518, 4182,...","[3895, 965, 3493, 3875, 7657, 8068, 7994, 6855...","[3895, 965, 3493, 3875, 7657, 8068, 7994, 6855...","[3895, 3132, 3493, 3002, 4426, 4773, 7994, 309...","[3895, 3132, 3493, 3002, 4426, 4773, 7994, 309..."
3,экология Белого моря,[6669],1,"[318, 8100, 7886, 6510, 7257, 6826, 3205, 8181...","[318, 8100, 7886, 6510, 7257, 6826, 3205, 8181...","[7886, 7553, 2204, 6508, 5925, 8232, 1174, 658...","[7886, 7553, 2204, 6508, 5925, 8232, 1174, 658...","[3331, 6510, 658, 6508, 7886, 7034, 7553, 3294...","[3331, 6510, 658, 6508, 7886, 7034, 7553, 3294..."
4,роботы для исследования Луны,[4005],1,"[4005, 5077, 2435, 1922, 3964, 6046, 1765, 272...","[4005, 5077, 2435, 1922, 3964, 6046, 1765, 272...","[4005, 2218, 2318, 7008, 4888, 2604, 3871, 166...","[4005, 2218, 2318, 7008, 4888, 2604, 3871, 166...","[4005, 651, 1922, 7134, 3964, 2732, 2218, 2661...","[4005, 651, 1922, 7134, 3964, 2732, 2218, 2661..."
...,...,...,...,...,...,...,...,...,...
3594,достижения российских ученых в области материа...,[7950],1,"[1902, 4869, 2142, 5002, 1530, 2429, 4812, 157...","[1902, 4869, 2142, 5002, 1530, 2429, 4812, 157...","[1530, 1576, 6023, 4628, 4869, 2006, 2968, 201...","[1530, 1576, 6023, 4628, 4869, 2006, 2968, 201...","[1530, 2968, 1576, 2018, 7510, 7260, 2006, 810...","[1530, 2968, 1576, 2018, 7510, 7260, 2006, 810..."
3595,антропология современного общества,[1342],1,"[2126, 1342, 8131, 4109, 7166, 3135, 7336, 294...","[2126, 1342, 8131, 4109, 7166, 3135, 7336, 294...","[1342, 2126, 1900, 4662, 7336, 5213, 4310, 535...","[1342, 2126, 1900, 4662, 7336, 5213, 4310, 535...","[1342, 4690, 4109, 6000, 4370, 1584, 4616, 147...","[1342, 4690, 4109, 6000, 4370, 1584, 4616, 147..."
3596,статьи про обновление исторического максимума ...,[270],1,"[270, 602, 6325, 4674, 5036, 543, 7040, 7898, ...","[270, 602, 6325, 4674, 5036, 543, 7040, 7898, ...","[270, 6845, 611, 5082, 4109, 539, 7056, 6138, ...","[270, 6845, 611, 5082, 4109, 539, 7056, 6138, ...","[270, 3758, 5149, 7056, 6325, 5591, 6138, 8192...","[270, 3758, 5149, 7056, 6325, 5591, 6138, 8192..."
3597,генетика структуры ДНК,[2130],1,"[6062, 5269, 7019, 6333, 5948, 1783, 2130, 151...","[6062, 5269, 7019, 6333, 5948, 1783, 2130, 151...","[2024, 6062, 6333, 7476, 7019, 2130, 1890, 777...","[2024, 6062, 6333, 7476, 7019, 2130, 1890, 777...","[6476, 1515, 5948, 7084, 7776, 7019, 679, 3624...","[6476, 1515, 5948, 7084, 7776, 7019, 679, 3624..."


In [ ]:
def recall(sug, rel):
  sug, rel = set(sug), set(rel)
  return len(sug & rel) / len(rel)

def mrr(sug, rel):
  for i, el in enumerate(sug, start=1):
    if el in rel:
      return 1 / i
  else:
    return 0

def eval_search(s, test_req):
  recall10 = test_req.apply(lambda row: recall(row[f'search_10_{s}'], row['id']), axis=1)
  recall20 = test_req.apply(lambda row: recall(row[f'search_20_{s}'], row['id']), axis=1)
  mrr10 = test_req.apply(lambda row: mrr(row[f'search_10_{s}'], row['id']), axis=1)
  mrr20 = test_req.apply(lambda row: mrr(row[f'search_20_{s}'], row['id']), axis=1)

  return {
      'recall@10': recall10.mean(),
      'recall@20': recall20.mean(),
      'MRR@10': mrr10.mean(),
      'MRR@20': mrr20.mean()
  }

In [ ]:
modes = ['none', 'rerank', 'other']
metrics = [eval_search(s, test_req=test_req) for s in modes]
results = pd.DataFrame(index=modes, data=metrics)
with pd.option_context('display.float_format', '{:.3f}'.format):
  print(results)

        recall@10  recall@20  MRR@10  MRR@20
none        0.569      0.641   0.410   0.415
rerank      0.538      0.623   0.387   0.393
other       0.501      0.597   0.319   0.326


# 3. Создание тренировочного датасета

In [ ]:
positive = train_req.explode('id').reset_index()[['request', 'id']]
positive['label'] = 1
positive

,request,id,label
0,последние новости SpaceX,4217,1
1,3D-моделирование,1579,1
2,3D-моделирование,3335,1
3,3D-моделирование,4831,1
4,3D-моделирование,7493,1
...,...,...,...
19098,использование узелкового письма в древних циви...,466,1
19099,статьи про улучшение чувствительности светящих...,3540,1
19100,автоматизация управления оборудованием,7106,1
19101,достижения в области искусственного интеллекта,8120,1


In [ ]:
tags_df = df.explode('tags').reset_index()
tags_list = list(tags_df['tags'].unique())
tags_lower = [s.lower() for s in tags_list]

def process_tag(s, tags_list=tags_list, tags_lower=tags_lower):
  pos = tags_lower.index(s.lower())
  return tags_list[pos]

tags_df['tags'] = tags_df['tags'].map(process_tag)
tags_df

,index,title,link,annot,tags
0,0,"Ученые обнаружили еще одну причину, по которой...",https://naked-science.ru/article/biology/uchen...,Голые землекопы живут почти 40 лет — невероятн...,Биология
1,0,"Ученые обнаружили еще одну причину, по которой...",https://naked-science.ru/article/biology/uchen...,Голые землекопы живут почти 40 лет — невероятн...,голые землекопы
2,0,"Ученые обнаружили еще одну причину, по которой...",https://naked-science.ru/article/biology/uchen...,Голые землекопы живут почти 40 лет — невероятн...,ДНК
3,0,"Ученые обнаружили еще одну причину, по которой...",https://naked-science.ru/article/biology/uchen...,Голые землекопы живут почти 40 лет — невероятн...,долголетие
4,1,"Бактерии создали из себя батарейку, чтобы разр...",https://naked-science.ru/article/biology/bacte...,Они использовали содержащееся в почве железо. ...,Биология
...,...,...,...,...,...
33291,8270,У берегов Западной Африки нашли древний ударны...,https://naked-science.ru/article/geology/udarn...,"Ученые допускают, что 66 миллионов лет назад с...",динозавры
33292,8271,Химики из «Сколтеха» объяснили стабильность уг...,https://naked-science.ru/article/chemistry/sko...,В ряду углеводородов среди множества соединени...,Химия
33293,8271,Химики из «Сколтеха» объяснили стабильность уг...,https://naked-science.ru/article/chemistry/sko...,В ряду углеводородов среди множества соединени...,органические молекулы
33294,8271,Химики из «Сколтеха» объяснили стабильность уг...,https://naked-science.ru/article/chemistry/sko...,В ряду углеводородов среди множества соединени...,органические соединения


In [ ]:
def sample_soft_neg(q, random_state=258, tags_df=tags_df):
  excluded_t = tags_df[tags_df['index'] == q]['tags']
  sampled_doc = tags_df[~ tags_df['tags'].isin(excluded_t)].sample(1, random_state=random_state)
  return int(sampled_doc.iloc[0, 0])

In [ ]:
soft_negatives = positive.copy()
soft_negatives['passage'] = soft_negatives['id'].map(sample_soft_neg)

In [ ]:
soft_negatives = soft_negatives[['request', 'passage', 'label']]
soft_negatives['label'] = 0
soft_negatives['passage'] = soft_negatives['passage'].map(lambda x: texts[x])

positive.rename(columns={'id': 'passage'}, inplace=True)
positive['passage'] = positive['passage'].map(lambda x: texts[x])

In [ ]:
emb_model = SentenceTransformer('cointegrated/rubert-tiny2', device=device)
dataset = Dataset.from_pandas(positive[['request', 'passage']])

hard_negatives = mine_hard_negatives(
    dataset,
    emb_model,
    anchor_column_name='request',
    positive_column_name='passage',
    num_negatives=1,
    range_min=10,
    range_max=100,
    sampling_strategy='top',
    output_format='labeled-pair',
)

hard_negatives = hard_negatives.to_pandas()
hard_negatives = hard_negatives[hard_negatives['label'] == 0].reset_index(drop=True)

Found 13407 unique queries out of 19103 total queries.
Found an average of 1.425 positives per query.


Batches:   0%|          | 0/253 [00:00<?, ?it/s]

Batches:   0%|          | 0/419 [00:00<?, ?it/s]

Negative candidates mined, preparing dataset...
Metric       Positive       Negative     Difference
Count          19,099         19,099               
Mean           0.5248         0.5709        -0.0461
Median         0.5382         0.5862        -0.0456
Std            0.1000         0.0749         0.0553
Min            0.2226         0.3681        -0.3110
25%            0.4547         0.5191        -0.0823
50%            0.5382         0.5862        -0.0456
75%            0.6003         0.6285        -0.0092
Max            0.8030         0.7462         0.1976
Could not find enough negatives for 4 samples (0.01%). Consider adjusting the range_max and range_min parameters if you'd like to find more valid negatives.


In [ ]:
final_train = pd.concat([positive, soft_negatives, hard_negatives], ignore_index=True)
final_train = final_train[~ final_train[['request', 'passage']].duplicated()]

path = '/content/drive/MyDrive/Colab Notebooks/OTUS_project/train.csv'
final_train.to_csv(path, index=False)

,request,passage,label
0,последние новости SpaceX,Компания SpaceX сегодня планирует провести тре...,1
1,3D-моделирование,"Как понять, сколько в нефтяных залежах осталос...",1
2,3D-моделирование,"Аэродинамика — один из важных факторов, влияющ...",1
3,3D-моделирование,Для создания трехмерных моделей (архитектурных...,1
4,3D-моделирование,"Объединенная команда Сколтеха, Института искус...",1
...,...,...,...
51608,процессы образования окаменелостей,Хиолиты — это проблемные окаменелости животных...,0
51609,использование узелкового письма в древних циви...,В IV веке нашей эры в Европе появились воинств...,0
51610,статьи про улучшение чувствительности светящих...,Продолжительные наблюдения за таймингом сигнал...,0
51611,автоматизация управления оборудованием,Ученые ПНИПУ создали универсальный цифровой тр...,0


# 4. Дообучение модели

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/OTUS_project/train.csv'
final_train = pd.read_csv(path).sample(frac=1, replace=False, random_state=869).reset_index(drop=True)
final_train

,request,passage,label
0,связь между сном на кровати родителей и смертн...,"Почти в 60% случаев, когда дети внезапно умира...",1
1,исследования в области почвоведения в Тюмени,"Химики доказали, что, вопреки распространенном...",0
2,геноцид в Скандинавии,"Авторы нового исследования считают, что выходц...",1
3,биотопливо,"Анализ исследований, в которых сравнивался эфф...",0
4,дискриминация,Пока в мире принимают «законы о согласии» и по...,1
...,...,...,...
51271,преимущества прикосновений для здоровья,"Исследование американских ученых, проведенное ...",0
51272,статьи о водоочистке,Химический состав древесины может указать на т...,0
51273,мифы о здоровых продуктах,"Считается, что трудолюбие и усердие могут комп...",0
51274,влияние быстрой лапши на здоровье,"По данным издания Gazeta.Ru, вопреки предрассу...",1


In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/OTUS_project/model_train'
train_dataset = Dataset.from_pandas(final_train)
pretrained_rr = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1', device=device)
loss = BinaryCrossEntropyLoss(model=pretrained_rr)
args_train = CrossEncoderTrainingArguments(
    output_dir=path,
    save_strategy='no',
    load_best_model_at_end=False,
    batch_sampler=BatchSamplers.GROUP_BY_LABEL,
    fp16=True,

    num_train_epochs=5,
    learning_rate=1e-5,
    per_device_train_batch_size=16,

    warmup_ratio=0.1,
    weight_decay=0.01,
    optim='adamw_torch',
    lr_scheduler_type='linear'
)

trainer = CrossEncoderTrainer(
    model=pretrained_rr,
    args=args_train,
    train_dataset=train_dataset,
    loss=loss
)

trainer.train()

Step,Training Loss
500,0.779100
1000,0.000100
1500,0.193900
2000,0.000100
2500,0.000000
3000,0.000000
3500,0.000000
4000,0.000000
4500,0.000000
5000,0.000000


In [ ]:
query = list(test_req['request'])
test_req['search_20_trained'] = semantic_search(query, retriever=model, reranker=pretrained_rr, top_k=20)
test_req['search_10_trained'] = test_req['search_20_trained'].map(lambda x: x[:10])

path = '/content/drive/MyDrive/Colab Notebooks/OTUS_project/final_search.csv'
test_req.to_csv(path, index=False)

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/OTUS_project/final_search.csv'
test_req = pd.read_csv(path)
test_req = test_req[~ test_req[['request', 'id']].duplicated()]

excl_cols = ['request', 'n']
list_cols = [col for col in test_req.columns if col not in excl_cols]
for col in list_cols:
  test_req[col] = test_req[col].map(json.loads)

modes = ['none', 'rerank', 'other', 'trained']
metrics = [eval_search(s, test_req=test_req) for s in modes]
results = pd.DataFrame(index=modes, data=metrics)
with pd.option_context('display.float_format', '{:.3f}'.format):
  print(results)

In [ ]:
pretrained_rr.save_pretrained('/content/drive/MyDrive/Colab Notebooks/OTUS_project/model_train')

# 5. Рекомендации

In [ ]:
def rec_from_search(to_search, search_result, retriever, reranker=None):
  rel_tags = list(df.iloc[search_result].explode('tags')['tags'].unique())
  pref_q = [f'query: {q}' for q in [to_search] + rel_tags]
  request = retriever.encode(
      pref_q,
      convert_to_tensor=True,
      device=device,
      normalize_embeddings=True
  )
  scores = request[1:] @ request[0]
  query = torch.argsort(scores, descending=True)[1].item()

  return semantic_search(rel_tags[query], retriever=model, reranker=pretrained_rr, top_k=10)[0]

In [ ]:
query = 'нейронные сети'
test_search = semantic_search(query, retriever=model, reranker=None)[0]
reccomends = rec_from_search(query, test_search, retriever=model, reranker=None)
df.iloc[test_search][['title']]

0it [00:00, ?it/s]

,title
7751,ИИ в стиле технопанка: создана механическая не...
6134,Нейросети: зачем они обычному человеку?
5605,Тюменские физики обучили нейронную сеть ассоци...
5334,Российские ученые с помощью нейросети усоверше...
1047,Российские ученые научились модифицировать бел...
6710,Ученые составили полную карту мозга дрозофилы
2658,Нейросеть ускорит «гаражную амнистию»
5600,"Ученые выяснили, как системы головного мозга р..."
4883,Ученые «персонализировали» подбор нейросети дл...
1247,Нейросети упростят сложные расчеты в науке и ф...


In [ ]:
df.iloc[reccomends][['title']]

,title
7206,Нейрообразование: от термина до рынка
2878,В мозге цыплят обнаружили специальные нейроны ...
6080,Исследование активности мозга помогло предсказ...
4393,Первый человек с чипом Neuralink смог управлят...
5334,Российские ученые с помощью нейросети усоверше...
1047,Российские ученые научились модифицировать бел...
2372,В МФТИ ускорили поиск новых материалов с задан...
8143,Молекулярный атлас мозга бородатой агамы расск...
2541,Альтернативный нейротех: от глии до искусствен...
4691,Математики описали синхронизирование колебаний...
